In [2]:
import argparse
import sys
import os
import time
import datetime
import ast
from collections import Counter
from pathlib import Path
import hydra

import numpy as np
import torch

# Ensure local src/ is importable
sys.path.insert(0, str(Path(os.getcwd()).parent / "src"))

from my_genQC.inference.eval_metrics import UnitaryFrobeniusNorm, UnitaryInfidelityNorm
from my_genQC.inference.evaluation_helper import get_unitaries, get_srvs
from my_genQC.inference.sampling import generate_compilation_tensors, generate_tensors, decode_tensors_to_backend
from my_genQC.pipeline.diffusion_pipeline import DiffusionPipeline
from my_genQC.platform.simulation import Simulator, CircuitBackendType
from my_genQC.platform.tokenizer.circuits_tokenizer import CircuitTokenizer
from my_genQC.utils.misc_utils import infer_torch_device, get_entanglement_bins
from my_genQC.dataset import circuits_dataset
from my_genQC.models.config_model import ConfigModel
from my_genQC.utils.config_loader import load_config, store_tensor, load_tensor

from quantum_diffusion.data.dataset import DatasetLoader
from quantum_diffusion.evaluation.evaluator import SRVEvaluator

In [3]:
# Config loading
with hydra.initialize(version_base=None, config_path="../conf"):
    cfg = hydra.compose(config_name="config.yaml", overrides=["evaluation=remote_model"])
    cfg = cfg["evaluation"]

In [4]:
def evaluate_remote_model(num_qubits: int, tensor_out_file, config):
    config.dataset = f"../datasets/paper_qiskit/srv_{num_qubits}q_dataset"
    config.wandb.run_name = f"{num_qubits}q_remote_model_evaluation"
    evaluator = SRVEvaluator(config=cfg)

    tensors_out = load_tensor(f"../scripts/inference/{tensor_out_file}", device="cuda")

    decoded_circuits = evaluator.decode_tensors(tensors_out)
    valid_indices, target_srvs, predicted_srvs = evaluator.validate_and_calculate_srvs(decoded_circuits)
    srv_exact_match_rate, acc_per_entanglement = evaluator.calculate_metrics(target_srvs, predicted_srvs)

    return evaluator, decoded_circuits, valid_indices, target_srvs, predicted_srvs, srv_exact_match_rate, acc_per_entanglement

## Quditkit dataset

In [1]:
q3_evaluator, q3_decoded_circuits, q3_valid_indices, q3_target_srvs, q3_predicted_srvs, q3_srv_exact_match_rate, q3_acc_per_entanglement = evaluate_remote_model(3, "3q_163327_samples.pt", cfg)

NameError: name 'evaluate_remote_model' is not defined

In [ ]:
q4_evaluator, q4_decoded_circuits, q4_valid_indices, q4_target_srvs, q4_predicted_srvs, q4_srv_exact_match_rate, q4_acc_per_entanglement = evaluate_remote_model(4, "4q_281970_samples.pt", cfg)

In [ ]:
q5_evaluator, q5_decoded_circuits, q5_valid_indices, q5_target_srvs, q5_predicted_srvs, q5_srv_exact_match_rate, q5_acc_per_entanglement = evaluate_remote_model(5, "5q_497496_samples.pt", cfg)

In [ ]:
q6_evaluator, q6_decoded_circuits, q6_valid_indices, q6_target_srvs, q6_predicted_srvs, q6_srv_exact_match_rate, q6_acc_per_entanglement = evaluate_remote_model(6, "6q_499926_samples.pt", cfg)

In [ ]:
q7_evaluator, q7_decoded_circuits, q7_valid_indices, q7_target_srvs, q7_predicted_srvs, q7_srv_exact_match_rate, q7_acc_per_entanglement = evaluate_remote_model(7, "7q_499968_samples.pt", cfg)

In [ ]:
q8_evaluator, q8_decoded_circuits, q8_valid_indices, q8_target_srvs, q8_predicted_srvs, q8_srv_exact_match_rate, q8_acc_per_entanglement = evaluate_remote_model(8, "8q_599936_samples.pt", cfg)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Map qubits -> your dicts (keys=x, values=y)
acc_by_qubits = {
    3: q3_acc_per_entanglement,
    4: q4_acc_per_entanglement,
    5: q5_acc_per_entanglement,
    6: q6_acc_per_entanglement,
    7: q7_acc_per_entanglement,
    8: q8_acc_per_entanglement,
}

fig, ax = plt.subplots(figsize=(5, 3.6), dpi=150)

# ---- Discrete colormap for integer qubits (stepped colorbar) ----
qubits = np.array(sorted(acc_by_qubits.keys()))
cmap = plt.get_cmap("Oranges", len(qubits)) # quantized version of Oranges

# boundaries between integers -> discrete steps
bounds = np.arange(qubits.min() - 0.5, qubits.max() + 1.5, 1.0)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

for q, acc in acc_by_qubits.items():
    # Ensure x is sorted (dict order may not match numeric order)
    xs = sorted(acc.keys())
    ys = [acc[x] for x in xs]

    ax.plot(
        xs, ys,
        marker="o", markersize=3,
        # linewidth=2,
        color=cmap(norm(q)),
        alpha=0.95,
    )

# ---- Stepped colorbar labeled "Qubits" ----
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, pad=0.02, ticks=qubits, boundaries=bounds)
cbar.set_label("Qubits")

# Styling to resemble the example
ax.set_ylim(0, 1.05)
ax.grid(True, alpha=0.35)

ax.set_ylabel("Accuracy")
ax.set_xlabel("Entangled qubits")

plt.tight_layout()
plt.show()


In [ ]:
import dill as pkl

for n_qubits, q_results in enumerate([[q3_valid_indices, q3_target_srvs, q3_predicted_srvs, q3_srv_exact_match_rate, q3_acc_per_entanglement],
                                      [q4_valid_indices, q4_target_srvs, q4_predicted_srvs, q4_srv_exact_match_rate, q4_acc_per_entanglement],
                                      [q5_valid_indices, q5_target_srvs, q5_predicted_srvs, q5_srv_exact_match_rate, q5_acc_per_entanglement],
                                      [q6_valid_indices, q6_target_srvs, q6_predicted_srvs, q6_srv_exact_match_rate, q6_acc_per_entanglement],
                                      [q7_valid_indices, q7_target_srvs, q7_predicted_srvs, q7_srv_exact_match_rate, q7_acc_per_entanglement],
                                      [q8_valid_indices, q8_target_srvs, q8_predicted_srvs, q8_srv_exact_match_rate, q8_acc_per_entanglement]]):

    path = f"./results/quditkit_dataset/q{n_qubits + 3}"
    os.makedirs(path, exist_ok=True)

    with open(os.path.join(path, f"q{n_qubits + 3}_valid_indices.pkl"), "wb") as f:
        pkl.dump(q_results[0], f)

    store_tensor(q_results[1], os.path.join(path, f"q{n_qubits + 3}_target_srvs.pt"))
    store_tensor(q_results[2], os.path.join(path, f"q{n_qubits + 3}_predicted_srvs.pt"))

    with open(os.path.join(path, f"q{n_qubits + 3}_srv_exact_match_rate.pkl"), "wb") as f:
        pkl.dump(q_results[3], f)

    with open(os.path.join(path, f"q{n_qubits + 3}_acc_per_entanglement.pkl"), "wb") as f:
        pkl.dump(q_results[4], f)
